In [5]:
%load_ext autoreload
%autoreload 2

In [60]:
import numpy as np
import torch 

In [55]:
H = 10
W = 10
focal = 138
# This pose is the camera facing the -x direction 
pose = torch.tensor([
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [-1, 0, 0, -1],
    [0, 0, 0, 1]
])

In [81]:
def yuzhen_get_rays(pose, H, W, focal):
    #the input is:
        # 1. the focal length (use the to calculate the relative direction of the ray according to the camera)
        # 2. the 4*4 camera position matreix: 
            # contain the 3*3 rotation matrix
            # 3*1 transfomation matrix
        # 3. the size of the frame (pixel as the unit in order to define the ray number)

    #the output is: 
        # 1. the original of the rays
        # 2. the direction of the rays 



    R = torch.zeros([W])
    for i in range(-W/2,W/2):
        R[i+W/2] = i
    print(R)

    C = torch.meshgrid(torch.arange(W))
    print(C)
    
    return 

R = torch.zeros([W])
for i in range(int(-W*0.5),int(W*0.5)):
    R[int(i+W*0.5)] = i
# print(R)

R = torch.unsqueeze(R,0) # create another row dimention 
# print(R)
# print(R.shape)
R = R.expand(H,-1)
# print(R)
# print(R.shape)
 

C = torch.zeros([H])
for i in range(int(-H/2), int(H/2)):
    C[int(i+int(H/2))] = -i
C = torch.unsqueeze(C,1)
# print(C)
C = C.expand(-1,W)
# print(C)

R = torch.unsqueeze(R,2)
print(R.shape)
C = torch.unsqueeze(C,2)
print(C.shape)
R_C = torch.cat((R, C), 2)
print(R_C.shape)
# print(R_C)

z_dim = torch.ones(R.shape)
z_dim = z_dim * -focal
print(z_dim.shape)
# print(z_dim)

R_C_Z = torch.cat((R_C, z_dim),2)
print(R_C_Z.shape)
R_C_Z = R_C_Z / focal
# print (R_C_Z)


rotation_matrix = pose[:3, :3]

## one direction for test demo--------
test_dir = R_C_Z[0][0]
print(test_dir)

 
real_test_dir = test_dir * rotation_matrix
print(rotation_matrix)
print (real_test_dir)
real_test_dir = torch.sum(real_test_dir, -1)
print(real_test_dir)
##--------------------------------------

## do it for all the direction---------
R_C_Z = torch.unsqueeze(R_C_Z,2) # change the shape from [10,10,3] to [10,10,1,3] used for the later matiplication 
print(R_C_Z.shape)
real_R_C_Z = R_C_Z * rotation_matrix # [10,10,1,3] * [3*3] --> [10,10,1,3] * [1,1,3,3] = [10,10,3,3]
real_R_C_Z = torch.sum(real_R_C_Z,-1) # add the colum of the 3*3 matrix together
print(real_R_C_Z.shape)
# print(real_R_C_Z)
##-------------------------------------


rays_direction = real_R_C_Z

##-----------------------start to calculate the rays_original-------------------------------
## the origin of the rays is the camera position --> the transportation 3D vector 
rays_origion = pose[:3, -1] # the :3 means the row 0,1,2. the -1 means the last colum 
print(rays_origion.shape)
# print(rays_origion)

## since later we need to use the rays_direction and rays_origion to do the calculation, so it's better
## to have them have the shape shape 
#rays_direction shape --> [10,10,3]
#rays_origion shape --> [3]

rays_origion = rays_origion.expand(rays_direction.shape)
print(rays_origion.shape)
print(rays_origion)


torch.Size([10, 10, 1])
torch.Size([10, 10, 1])
torch.Size([10, 10, 2])
torch.Size([10, 10, 1])
torch.Size([10, 10, 3])
tensor([-0.0362,  0.0362, -1.0000])
tensor([[ 0,  0,  1],
        [ 0,  1,  0],
        [-1,  0,  0]])
tensor([[-0.0000,  0.0000, -1.0000],
        [-0.0000,  0.0362, -0.0000],
        [ 0.0362,  0.0000, -0.0000]])
tensor([-1.0000,  0.0362,  0.0362])
torch.Size([10, 10, 1, 3])
torch.Size([10, 10, 3])
torch.Size([3])
torch.Size([10, 10, 3])
tensor([[[ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1]],

        [[ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1]],

        [[ 0,  0, -1],
         [ 0,  0, -1],
         [ 0,  0, -1],
     

In [ ]:
def yuzhen_render():
    #the input is 
        # 1. the camera position
        # 2. ray direction 
        # 3. the near point 
        # 4. the far point
        # 5. the sample point number

    #the overfitted mode: imported

    #the output is 
        # 1. the density at each point
        # 2. the RGB color at each point 


In [69]:
def get_rays(H, W, focal, pose):
  """
  This function generates camera rays for each pixel in an image. It calculates the origin and direction of rays
  based on the camera's intrinsic parameters (focal length) and extrinsic parameters (pose).
  The rays are generated in world coordinates, which is crucial for the NeRF rendering process.

  Parameters:
  H (int): Height of the image in pixels.
  W (int): Width of the image in pixels.
  focal (float): Focal length of the camera.
  pose (torch.Tensor): Camera pose matrix of size 4x4.

  Returns:
  tuple: A tuple containing two elements:
      rays_o (torch.Tensor): Origins of the rays in world coordinates.
      rays_d (torch.Tensor): Directions of the rays in world coordinates.
  """
  # Create a meshgrid of image coordinates (i, j) for each pixel in the image.
  i, j = torch.meshgrid(
      torch.arange(W, dtype=torch.float32),
      torch.arange(H, dtype=torch.float32)
  )
  # print("i is:", i)
  # print("j is:", j)
  i = i.t()
  j = j.t()
  # print("i is:", i)
  # print("j is:", j)

  # Calculate the direction vectors for each ray originating from the camera center.
  # We assume the camera looks towards -z.
  # The coordinates are normalized with respect to the focal length.
  dirs = torch.stack(
      [(i - W * 0.5) / focal,
        -(j - H * 0.5) / focal,
        -torch.ones_like(i)], -1
      )

  # Transform the direction vectors (dirs) from camera coordinates to world coordinates.
  # This is done using the rotation part (first 3 columns) of the pose matrix.
  rays_d = torch.sum(dirs[..., np.newaxis, :] * pose[:3, :3], -1) #[100,100,1,3] * [3,3] = [100,100,3,3] after sum(,-1) --> [100,100,3]

  # The ray origins (rays_o) are set to the camera position, given by the translation part (last column) of the pose matrix.
  # The position is expanded to match the shape of rays_d for broadcasting.
  rays_o = pose[:3, -1].expand(rays_d.shape)

  # Return the origins and directions of the rays.
  return rays_o, rays_d




i, j = torch.meshgrid(
    torch.arange(W, dtype=torch.float32),
    torch.arange(H, dtype=torch.float32)
)
# print("i is:", i)
# print("j is:", j)
i = i.t()
j = j.t()
# print("i is:", i)
# print("j is:", j)

# Calculate the direction vectors for each ray originating from the camera center.
# We assume the camera looks towards -z.
# The coordinates are normalized with respect to the focal length.
dirs = torch.stack(
    [(i - W * 0.5) / focal,
      -(j - H * 0.5) / focal,
      -torch.ones_like(i)], -1
    )
# print(dirs.shape)
# print(dirs)
rays_d = torch.sum(dirs[..., np.newaxis, :] * pose[:3, :3], -1)  
print(rays_d.shape)
print(rays_d)

 



torch.Size([10, 10, 3])
tensor([[[-1.0000,  0.0362,  0.0362],
         [-1.0000,  0.0362,  0.0290],
         [-1.0000,  0.0362,  0.0217],
         [-1.0000,  0.0362,  0.0145],
         [-1.0000,  0.0362,  0.0072],
         [-1.0000,  0.0362,  0.0000],
         [-1.0000,  0.0362, -0.0072],
         [-1.0000,  0.0362, -0.0145],
         [-1.0000,  0.0362, -0.0217],
         [-1.0000,  0.0362, -0.0290]],

        [[-1.0000,  0.0290,  0.0362],
         [-1.0000,  0.0290,  0.0290],
         [-1.0000,  0.0290,  0.0217],
         [-1.0000,  0.0290,  0.0145],
         [-1.0000,  0.0290,  0.0072],
         [-1.0000,  0.0290,  0.0000],
         [-1.0000,  0.0290, -0.0072],
         [-1.0000,  0.0290, -0.0145],
         [-1.0000,  0.0290, -0.0217],
         [-1.0000,  0.0290, -0.0290]],

        [[-1.0000,  0.0217,  0.0362],
         [-1.0000,  0.0217,  0.0290],
         [-1.0000,  0.0217,  0.0217],
         [-1.0000,  0.0217,  0.0145],
         [-1.0000,  0.0217,  0.0072],
         [-1.0000,  0.